# Llama 3.2 Vision: Independent Single-Turn Bank Statement Extraction

**Protocol**: Three independent single-turn prompts (NOT multi-turn conversation)

**Key Insight**: Multi-turn conversation degrades accuracy due to context accumulation. Each turn runs with fresh context.

---

## Complete Workflow

```
Turn 0: Image + Prompt → Headers (fresh context)
        ↓ (Python pattern matching)
Turn 1: Image + Prompt → Full Table (fresh context, uses headers from Turn 0)
        ↓ (Python parsing)  
Turn 2: Image + Prompt → Filtered Table (fresh context, uses headers from Turn 0)
        ↓ (Python schema extraction)
```

### Pipeline Stages:
1. **Turn 0 (Independent)**: Image + prompt to identify column headers
2. **Pattern Matching (Python)**: Map headers to generic concepts (Date, Description, Debit)
3. **Turn 1 (Independent)**: Image + prompt to extract full table using identified headers
4. **Turn 2 (Independent)**: Image + prompt to filter withdrawal transactions
5. **Python Parsing**: Convert markdown to schema format

### Key Differences from Multi-Turn:
- ❌ **No conversation history** - each turn starts fresh
- ❌ **No context accumulation** - prevents attention dilution
- ✅ **Direct image access** - every turn has full image attention
- ✅ **Better accuracy** - empirically proven to work better than multi-turn

In [ ]:
# Cell 1: Imports

from pathlib import Path
import random

import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration

# Set Random Seed for Reproducibility

In [ ]:
# Cell 3: Set random seed

from common.reproducibility import set_seed
set_seed(42)

✅ Random seed set to 42 for reproducibility


# Load the model

In [ ]:
# Cell 5: Load Llama-3.2-Vision model
# Update this path to your local Llama model
# model_id = "/home/jovyan/shared_PTM/Llama-3.2-11B-Vision-Instruct"
model_id = "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct"

print("🔧 Loading Llama-3.2-Vision model...")
# model = MllamaForConditionalGeneration.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# processor = AutoProcessor.from_pretrained(model_id)

from common.llama_model_loader_robust import load_llama_model_robust

model, processor = load_llama_model_robust(
    model_path=model_id,
    use_quantization=False,
    device_map='auto',
    max_new_tokens=2000,
    torch_dtype='bfloat16',
    low_cpu_mem_usage=True,
    verbose=True
)

# Add tie_weights() call
try:
    model.tie_weights()
    print("✅ Model weights tied successfully")
except Exception as e:
    print(f"⚠️ tie_weights() warning: {e}")

# processor

# Load the image

In [ ]:
# Cell 7: Load bank statement image
# Update this path to your test image
# imageName = "/home/jovyan/shared_PoC_data/evaluation_data/synthetic_bank_images/cba_amount_balance.png"
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/images/image_008.png"
print("📁 Loading image...")
image = Image.open(imageName)

# CRITICAL: Store as list for multi-turn compatibility
images = [image]

print(f"✅ Image loaded: {image.size}")
print(f"✅ Images list created with {len(images)} image(s)")

# Multi-Turn Bank Statement Protocol
- Turn 0: Identify actual table headers
- Turn 1: Extract full table using those headers
- Turn 2: Filter using the actual column names found

In [ ]:
# Cell 9: Turn 0 - Identify table headers (prompt)
# TURN 0: Identify Table Headers
# First, identify the actual column headers used in this specific bank statement

prompt = """
Look at the transaction table in this bank statement image.

IMPORTANT STRUCTURAL NOTE:
Some bank statements show dates as section headings with multiple transactions underneath.
If you see this structure, remember that each transaction needs its explicit date in the final output.

What are the exact column header names used in the transaction table?

List each column header exactly as it appears, in order from left to right.
Do not interpret or rename them - use the EXACT text from the image.
"""

# Create message structure for Llama
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": prompt,
            },
        ],
    }
]

print("💬 TURN 0: Identifying actual table headers")
print("🤖 Generating response with Llama-3.2-Vision...")

# Process the prompt

In [ ]:
# Cell 11: Turn 0 - Execute and parse headers
# Process the input using the CORRECT multi-turn pattern
# Based on: https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa

textInput = processor.apply_chat_template(
    messageDataStructure, add_generation_prompt=True
)

# CRITICAL: Use named parameter 'images=' with list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response with deterministic parameters
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# CRITICAL: Trim input tokens from output (this is the key to clean responses!)
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("✅ Response generated successfully!")
print("\n" + "=" * 60)
print("TURN 0 - IDENTIFIED TABLE HEADERS:")
print("=" * 60)
print(cleanedOutput)
print("=" * 60)

# CRITICAL: Parse the identified headers for use in subsequent turns
# Extract column names from the response
header_lines = [line.strip() for line in cleanedOutput.split('\n') if line.strip()]
identified_headers = []

# Look for numbered list or bullet points
for line in header_lines:
    # Remove common list markers
    cleaned = line.lstrip('0123456789.-•* ').strip()
    
    # Strip markdown bold formatting
    cleaned = cleaned.replace('**', '').replace('__', '')
    
    # Skip section headers (lines ending with colon)
    if cleaned.endswith(':'):
        continue
    
    # Skip long sentences (likely explanatory text, not headers)
    if len(cleaned) > 40:
        continue
        
    if cleaned and len(cleaned) > 2:  # Ignore very short strings
        identified_headers.append(cleaned)

print(f"\n📋 Parsed {len(identified_headers)} column headers:")
for i, header in enumerate(identified_headers, 1):
    print(f"  {i}. '{header}'")

# Store headers for use in subsequent turns
table_headers = identified_headers

# Save the table headers
output_path = Path("llama_table_headers.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput)

print(f"\n✅ Table headers saved to: {output_path}")
print("💡 These LITERAL header names will be used in Turn 1 & 2 prompts")

## Pattern Matching: Map Generic Concepts to Actual Headers

Different bank statements use different column names. Use pattern matching to identify:
- Which header represents **Date**
- Which header represents **Description/Details**  
- Which header represents **Debit/Withdrawal**

In [ ]:
# Cell 13: Pattern Matching - Map headers to generic columns
# Pattern Matching: Map extracted headers to generic concepts
# This handles variety in bank statement column naming conventions

# Pattern keywords for each concept (in priority order)
DATE_PATTERNS = ['date', 'day', 'transaction date', 'trans date']
DESCRIPTION_PATTERNS = [
    'description', 'details', 'transaction details', 'trans details',
    'particulars', 'narrative', 'transaction', 'trans'
]
DEBIT_PATTERNS = ['debit', 'withdrawal', 'withdrawals', 'paid', 'paid out', 'spent', 'dr']
CREDIT_PATTERNS = ['credit', 'deposit', 'deposits', 'received', 'cr']
BALANCE_PATTERNS = ['balance', 'bal', 'running balance']

# NEW: Pattern for single-column transaction formats (e.g., "Amount" instead of separate Debit/Credit)
AMOUNT_PATTERNS = ['amount', 'amt', 'value', 'total']

def match_header(headers, patterns, fallback=None):
    """Match a header using pattern keywords.
    
    Matching strategy:
    1. Exact match (case-insensitive)
    2. Substring match (only for patterns with length > 2 to avoid false positives)
    """
    headers_lower = [h.lower() for h in headers]
    
    # Try exact match first
    for pattern in patterns:
        for i, header_lower in enumerate(headers_lower):
            if pattern == header_lower:
                return headers[i]
    
    # Try substring match (only for patterns longer than 2 chars)
    for pattern in patterns:
        if len(pattern) > 2:  # Avoid false positives like 'cr' matching 'description'
            for i, header_lower in enumerate(headers_lower):
                if pattern in header_lower:
                    return headers[i]
    
    return fallback

# Perform pattern matching on extracted headers
date_col = match_header(table_headers, DATE_PATTERNS, fallback=table_headers[0] if table_headers else 'Date')
desc_col = match_header(table_headers, DESCRIPTION_PATTERNS, fallback=table_headers[1] if len(table_headers) > 1 else 'Description')

# NEW: First try to match a generic "Amount" column (for 4-column formats)
amount_col = match_header(table_headers, AMOUNT_PATTERNS, fallback=None)

# Use amount_col as fallback if no separate debit/credit columns exist
# This handles formats like: Date | Description | Amount | Balance
debit_col = match_header(table_headers, DEBIT_PATTERNS, fallback=amount_col if amount_col else 'Debit')
credit_col = match_header(table_headers, CREDIT_PATTERNS, fallback=amount_col if amount_col else 'Credit')
balance_col = match_header(table_headers, BALANCE_PATTERNS, fallback='Balance')

print("=" * 60)
print("PATTERN MATCHING RESULTS:")
print("=" * 60)
print(f"📋 Extracted Headers: {table_headers}")
print(f"\n🔍 Mapped Columns:")
print(f"  Date        → '{date_col}'")
print(f"  Description → '{desc_col}'")
print(f"  Debit       → '{debit_col}'")
print(f"  Credit      → '{credit_col}'")
print(f"  Balance     → '{balance_col}'")
if amount_col:
    print(f"\n💡 Single-column format detected: '{amount_col}' used for both debit and credit")
print("=" * 60)
print("\n✅ These literal column names will be used in Turn 1 and Turn 2")
print("💡 Adjust patterns above if matching fails for your bank statement format")

### 🔑 Independent Single-Turn Pattern (NOT Multi-Turn Conversation)

**CRITICAL INSIGHT**: Multi-turn conversation accumulates context and degrades accuracy.

Instead, we use **three independent single-turn prompts**, each with fresh context:

#### Key Principles:

1. **No Conversation History**: Each turn is completely independent
2. **Fresh Image Attention**: Each turn processes the image directly
3. **No Context Accumulation**: Prevents attention dilution
4. **Headers as Parameters**: Turn 0 headers are passed as text parameters to Turn 1 and Turn 2

#### Message Structure for Each Turn:

Every turn uses the same fresh structure:
```python
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "<prompt with headers from Turn 0>"}
        ]
    }
]
```

**No assistant responses in history. No conversation accumulation.**

#### Why This Works Better:

- **Turn 0**: Clean context → accurate header identification
- **Turn 1**: Clean context + headers → accurate table extraction  
- **Turn 2**: Clean context + headers → accurate filtering

Each turn has **full attention** on the image, not diluted by conversation history.

In [ ]:
# Cell 15: NO conversation history (independent turns)
# 
# CRITICAL: We do NOT use conversation history in this notebook.
# Each turn is completely independent with fresh context.
#
# Why? Multi-turn conversation accumulates context and degrades accuracy:
# - Turn 0: 50 tokens → accurate
# - Turn 1 with history: 350 tokens → attention diluted → less accurate
# - Turn 2 with history: 2000 tokens → attention heavily diluted → poor accuracy
#
# Instead: Each turn starts fresh with just image + prompt

print("✅ Independent turn approach - NO conversation history")
print("💡 Each turn will have fresh context with direct image access")

### TURN 1: Extract Full Table in Markdown

Now that we know the actual column headers, extract the complete table:

In [ ]:
# Cell 17: Turn 1 - Extract full table (INDEPENDENT, fresh context)

# Build the header string using LITERAL names from Turn 0
header_string = " | ".join(table_headers)

follow_up_prompt = f"""
Extract the entire transaction table from this bank statement image in markdown format.

Use these EXACT column headers in this order:
{header_string}

Format requirements:
- Standard markdown table syntax with | delimiters
- Header row: | {header_string} |
- Separator row: | {" | ".join(["---"] * len(table_headers))} |

CRITICAL EXTRACTION RULES:
1. Extract EVERY transaction as a separate row
2. Each transaction MUST have its explicit date in the date column
3. If multiple transactions share a date heading, repeat that date for each transaction row
4. Do NOT skip or combine any rows
5. Keep all amounts with decimal values intact
6. Do NOT add explanatory text - only output the markdown table

CRITICAL COLUMN POSITION RULES:
7. Place each amount in the EXACT column where it appears in the original image
8. Do NOT move amounts between columns based on transaction type
9. Do NOT interpret whether a transaction should be a debit or credit
10. If an amount appears under the "{debit_col}" column in the image, put it in the {debit_col} column in your table
11. If an amount appears under the "{credit_col}" column in the image, put it in the {credit_col} column in your table
12. If a column is empty for a row in the image, leave it empty in your table
13. Preserve the EXACT visual layout - read amounts strictly by their column position

Example: If you see:
  01/06/2024
    Transaction A    $100
    Transaction B    $50
    
Output as TWO rows:
  | 01/06/2024 | Transaction A | $100 | ... |
  | 01/06/2024 | Transaction B | $50  | ... |
"""

# CRITICAL: Create FRESH message structure (NOT appending to conversation history)
messageDataStructure_turn1 = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": follow_up_prompt}
        ]
    }
]

print(f"💬 TURN 1: Extract full markdown table (INDEPENDENT)")
print(f"📋 Using literal headers: {table_headers}")
print(f"💡 Column position rules reference: '{debit_col}' and '{credit_col}'")
print(f"🔄 Fresh context (no conversation history)")
print("🤖 Generating response with Llama-3.2-Vision...")

# Process with FRESH context
textInput = processor.apply_chat_template(
    messageDataStructure_turn1, add_generation_prompt=True
)

# CRITICAL: Use named parameter 'images=' with list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# CRITICAL: Trim input tokens from output
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput2 = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("\n✅ Follow-up response generated successfully!")
print("\n" + "=" * 60)
print("TURN 1 - FULL MARKDOWN TABLE:")
print("=" * 60)
print(cleanedOutput2)
print("=" * 60)

# Save the markdown table
output_path = Path("llama_markdown_table_extraction.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput2)

print(f"\n✅ Markdown table saved to: {output_path}")
print("💡 Each transaction has explicit date, even if grouped by date heading")

### TURN 2: Filter Using Actual Column Names

Filter the extracted table using the specific column names identified in Turn 0:

In [ ]:
# Cell 19: Turn 2 - Filter withdrawals (INDEPENDENT, fresh context)

follow_up_prompt_3 = f"""
Extract ONLY the withdrawal/debit transactions from this bank statement image in markdown format.

Use these EXACT column names:
- {date_col}
- {desc_col}  
- {debit_col}

Filter rules:
- Only include rows where '{debit_col}' column has a value (not empty)
- Exclude credit/deposit transactions  
- Keep the markdown table format with header: | {date_col} | {desc_col} | {debit_col} |

Output only the filtered markdown table showing withdrawal transactions.
"""

# CRITICAL: Create FRESH message structure (NOT appending to conversation history)
messageDataStructure_turn2 = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": follow_up_prompt_3}
        ]
    }
]

print(f"💬 TURN 2: Filter withdrawal transactions (INDEPENDENT)")
print(f"📋 Filter columns: '{date_col}' | '{desc_col}' | '{debit_col}'")
print(f"🔄 Fresh context (no conversation history)")
print("🤖 Generating response with Llama-3.2-Vision...")

# Process with FRESH context
textInput = processor.apply_chat_template(
    messageDataStructure_turn2, add_generation_prompt=True
)

# Use named parameter 'images=' with list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# Trim input tokens from output
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput3 = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("\n✅ Follow-up response generated successfully!")
print("\n" + "=" * 60)
print("TURN 2 - FILTERED WITHDRAWALS:")
print("=" * 60)
print(cleanedOutput3)
print("=" * 60)

# Save filtered results
output_path = Path("llama_filtered_withdrawals.txt")
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput3)

print(f"\n✅ Filtered table saved to: {output_path}")
print(f"\n✅ Complete Protocol:")
print(f"   Turn 0: Identify headers → {table_headers}")
print(f"   Pattern Match: Date='{date_col}', Desc='{desc_col}', Debit='{debit_col}'")
print(f"   Turn 1: Extract full table (independent, fresh context)")
print(f"   Turn 2: Filter withdrawals (independent, fresh context)")
print("\n✅ Independent turns - better accuracy than multi-turn conversation!")

## Extract Schema Fields from Markdown Table

In [ ]:
# Cell 21: Parse markdown and extract schema fields
import re
from datetime import datetime

def parse_markdown_table(markdown_text):
    """Parse markdown table into list of dictionaries."""
    lines = [line.strip() for line in markdown_text.strip().split('\n') if line.strip()]
    
    # Find header row (first line with pipes)
    header_idx = None
    for i, line in enumerate(lines):
        if '|' in line and not set(line.replace('|', '').replace('-', '').strip()) == set():
            header_idx = i
            break
    
    if header_idx is None:
        return []
    
    # Parse headers
    header_line = lines[header_idx]
    headers = [h.strip() for h in header_line.split('|') if h.strip()]
    
    # Parse data rows (skip header and separator)
    rows = []
    for line in lines[header_idx + 1:]:
        if '|' in line:
            # Skip separator rows (contain only pipes, hyphens, and spaces)
            cleaned = line.replace('|', '').replace('-', '').replace(' ', '')
            if not cleaned:
                continue
            
            values = [v.strip() for v in line.split('|') if v.strip()]
            if len(values) == len(headers):
                rows.append(dict(zip(headers, values)))
    
    return rows

def extract_schema_fields(rows, date_col, desc_col, debit_col):
    """Extract fields in universal.yaml schema format."""
    if not rows:
        return {
            'TRANSACTION_DATES': 'NOT_FOUND',
            'LINE_ITEM_DESCRIPTIONS': 'NOT_FOUND',
            'TRANSACTION_AMOUNTS_PAID': 'NOT_FOUND',
            'STATEMENT_DATE_RANGE': 'NOT_FOUND'
        }
    
    # Extract lists
    dates = []
    descriptions = []
    amounts = []
    
    for row in rows:
        date = row.get(date_col, '').strip()
        desc = row.get(desc_col, '').strip()
        amount = row.get(debit_col, '').strip()
        
        if date:
            dates.append(date)
        if desc:
            descriptions.append(desc)
        if amount:
            amounts.append(amount)
    
    # Calculate statement date range
    date_range = 'NOT_FOUND'
    if dates:
        try:
            # Try to parse dates to find min/max
            parsed_dates = []
            for d in dates:
                # Try common formats
                for fmt in ['%d/%m/%Y', '%d/%m/%y', '%Y-%m-%d', '%d-%m-%Y']:
                    try:
                        parsed_dates.append(datetime.strptime(d, fmt))
                        break
                    except ValueError:
                        continue
            
            if parsed_dates:
                min_date = min(parsed_dates)
                max_date = max(parsed_dates)
                date_range = f"{min_date.strftime('%d/%m/%Y')} to {max_date.strftime('%d/%m/%Y')}"
        except Exception:
            # If parsing fails, use first and last date as-is
            date_range = f"{dates[0]} to {dates[-1]}"
    
    return {
        'TRANSACTION_DATES': ' | '.join(dates) if dates else 'NOT_FOUND',
        'LINE_ITEM_DESCRIPTIONS': ' | '.join(descriptions) if descriptions else 'NOT_FOUND',
        'TRANSACTION_AMOUNTS_PAID': ' | '.join(amounts) if amounts else 'NOT_FOUND',
        'STATEMENT_DATE_RANGE': date_range
    }

# Parse the filtered markdown table from Turn 2
parsed_rows = parse_markdown_table(cleanedOutput3)

print(f"📊 Parsed {len(parsed_rows)} transactions from markdown table")
print("\n" + "=" * 60)
print("SAMPLE ROWS:")
print("=" * 60)
for i, row in enumerate(parsed_rows[:3], 1):
    print(f"\nRow {i}:")
    for key, value in row.items():
        print(f"  {key}: {value}")

# Extract schema fields using the LITERAL column names from pattern matching
schema_fields = extract_schema_fields(parsed_rows, date_col, desc_col, debit_col)

print("\n" + "=" * 60)
print("EXTRACTED SCHEMA FIELDS:")
print("=" * 60)
for field, value in schema_fields.items():
    print(f"{field}: {value}")
print("=" * 60)

# Save to file
output_path = Path("llama_extracted_fields.txt")
with output_path.open("w", encoding="utf-8") as f:
    for field, value in schema_fields.items():
        f.write(f"{field}: {value}\n")

print(f"\n✅ Schema fields saved to: {output_path}")
print(f"💡 Fields extracted from columns: '{date_col}' | '{desc_col}' | '{debit_col}'")